In [ ]:
%pip install splunklib
%pip install pandasql
%pip install pandas

In [74]:
import splunklib.client as client
import splunklib.results as results
import pandas as pd
import time
from pandasql import sqldf
from sys import displayhook


pd.set_option('display.max_colwidth', None)

service = client.connect(  # Connect to Splunk
    host='192.168.28.132',
    port='8089',
    username='admin',
    password='Changeme1!',
    scheme='https')

In [75]:
# Modify your search query to output results in json
searchquery = "search (earliest=-72h index=windows EventCode=4663 AND NOT Process_Name IN (\"C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe\",\"*dllhost.exe\",\"*explorer.exe\",\"*MsMpEng.exe\",\"*firefox.exe\")) | regex Object_Name=\"(?i)^.*(login\sdata|local\sstate|profiles.ini|logins.json|key4.db|(cookies(\.sqliste)?))$\" | rex field=Message \"(?i)(?<object_operation>an\sattempt\swas\smade\sto.*)\" | eval processId = tonumber(Process_ID, 16) " 

# Create the job with the modified search query
job = service.jobs.create(searchquery)
while not job.is_done():
    time.sleep(2)

# Use the job's results, read into a JSON object
json_results = job.results(output_mode='json')

# Use the JSONResultsReader to parse the JSON results
rr = results.JSONResultsReader(json_results)

data = []

for result in rr:
    if isinstance(result, dict):
        data.append(result)

df_EID4663 = pd.DataFrame(data) # Convert the data to pandas DataFrame and display

In [76]:
searchquery = "search (earliest=-72h index=windows EventCode=4656 AND NOT Process_Name IN (\"C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe\",\"*dllhost.exe\",\"*explorer.exe\",\"*MsMpEng.exe\",\"*firefox.exe\")) | regex Object_Name=\"(?i)^.*(login\sdata|local\sstate|profiles.ini|logins.json|key4.db|(cookies(\.sqliste)?))$\" | rex field=Message \"(?i)(?<handle_operation>a\shandle\sto\san\sobject.*)\" | eval processId = tonumber(Process_ID, 16) " 

job = service.jobs.create(searchquery)
while not job.is_done():
    time.sleep(2)

json_results = job.results(output_mode='json')

rr = results.JSONResultsReader(json_results)

data2 = []

for result in rr:
    if isinstance(result, dict):
        data2.append(result)

df_EID4656 = pd.DataFrame(data2) 

In [77]:
df2_EID4656 = df_EID4656[['_time','host','handle_operation', 'processId', 'Process_Name','Object_Name']] # Select only the columns we want to display
df2_EID4663 = df_EID4663[['_time','host','object_operation', 'processId', 'Process_Name','Object_Name']] 

In [78]:
df3_EID4656 = pd.DataFrame(df2_EID4656) # Create a new dataframe
df3_EID4663 = pd.DataFrame(df2_EID4663) # Create a new dataframe

In [79]:
objects_and_handles = sqldf("SELECT df3_EID4663.host, df3_EID4663.at_time df3_EID4663.processId, df3_EID4663.Process_Name, df3_EID4663.Object_Name, df3_EID4656.handle_operation AS handle_request, df3_EID4663.object_operation AS object_access FROM df3_EID4656 INNER JOIN df3_EID4663 ON df3_EID4656.processId = df3_EID4663.processId") # Join the two dataframes on the processId column

In [80]:
objects_and_handles = objects_and_handles.drop_duplicates()

In [81]:
displayhook(objects_and_handles) # Display the combined dataframe

,host,processId,Process_Name,Object_Name,handle_request,object_access
0,ASGARD-WRKSTN,788,C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\python3.11.exe,C:\Users\loki\AppData\Roaming\Mozilla\Firefox\Profiles\8w9t1q1p.default-release\key4.db,A handle to an object was requested.,An attempt was made to access an object.
1,ASGARD-WRKSTN,788,C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\python3.11.exe,C:\Users\loki\AppData\Roaming\Mozilla\Firefox\Profiles\8w9t1q1p.default-release\logins.json,A handle to an object was requested.,An attempt was made to access an object.
3,ASGARD-WRKSTN,788,C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\python3.11.exe,C:\Users\loki\AppData\Roaming\Mozilla\Firefox\profiles.ini,A handle to an object was requested.,An attempt was made to access an object.
25,ASGARD-WRKSTN,2400,C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\python3.11.exe,C:\Users\loki\AppData\Roaming\Mozilla\Firefox\profiles.ini,A handle to an object was requested.,An attempt was made to access an object.
29,ASGARD-WRKSTN,972,C:\Users\loki\Desktop\SharpChrome.exe,C:\Users\loki\AppData\Local\Google\Chrome\User Data\Default\Login Data,A handle to an object was requested.,An attempt was made to access an object.
30,ASGARD-WRKSTN,972,C:\Users\loki\Desktop\SharpChrome.exe,C:\Users\loki\AppData\Local\Google\Chrome\User Data\Local State,A handle to an object was requested.,An attempt was made to access an object.
33,ASGARD-WRKSTN,2992,C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\python3.11.exe,C:\Users\loki\AppData\Roaming\Mozilla\Firefox\profiles.ini,A handle to an object was requested.,An attempt was made to access an object.
37,ASGARD-WRKSTN,5568,C:\Users\loki\Desktop\SharpChrome.exe,C:\Users\loki\AppData\Local\Google\Chrome\User Data\Default\Login Data,A handle to an object was requested.,An attempt was made to access an object.
38,ASGARD-WRKSTN,5568,C:\Users\loki\Desktop\SharpChrome.exe,C:\Users\loki\AppData\Local\Google\Chrome\User Data\Local State,A handle to an object was requested.,An attempt was made to access an object.
41,ASGARD-WRKSTN,3652,C:\Users\loki\Desktop\SharpChrome.exe,C:\Users\loki\AppData\Local\Microsoft\Edge\User Data\Default\Login Data,A handle to an object was requested.,An attempt was made to access an object.
